# Preliminaries

This notebook lets you import a gym environment and set up an agent that acts within the environment. Your tasks is to then implement some of the classical RL algorithms: Value iteration and Policy iteration. Play attention to how you are going to evaluate your agents.

First, we make sure that all dependencies are met

In [1]:
!pip install gym > /dev/null 2>&1

# Testing the Gym environments

Our next step is to import the gym package, create an environment, and make sure that we can use it.

In [2]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#create a cliff-walker
env = gym.make('CliffWalking-v0')

#set the start state
state = env.reset()
#and take some random actions
for i in range(4):
  #render the environment
  env.render()
  
  #select a random action
  action = env.action_space.sample()
  #take a step and record next state, reward and termination
  state, reward, done, _ = env.step(action)
  print("Acted: {}".format(action))
  print("State: {}".format(state))
  print("Reward: {}".format(reward))
  if done:
    #this environment only terminates once the goal is reached
    print("Done.")
    break

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Acted: 1
State: 36
Reward: -100
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Acted: 2
State: 36
Reward: -1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Acted: 0
State: 24
Reward: -1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Acted: 0
State: 12
Reward: -1


# Defining an agent

The next step is to define a class for our agents. We will derive from this class to later implement a Value Iteration, Policy Iteration and Monte Carlo control agent. The base class will only provide simple functionality.

In [4]:
class Agent :
  def __init__(self,env,discount_factor):
    self.env = env
    self.gamma = discount_factor
  
  def act(self, state):
    return self.env.action_space.sample() #returns a random action

  def evaluate(self):
    # now let's test our random action agent
    n_steps = 100 #number of steps per episode

    s = env.reset()
    episode_reward = 0
    
    for i in range(n_steps):
      s, r, d, _ = env.step(self.act(s))
      episode_reward += r
      if d:
        break
    return episode_reward

#test simple evaluation function
random_agent = Agent(env,0.99)
episode_reward=random_agent.evaluate()
print("Episode return {}".format(episode_reward))

Episode return -1387


# Value Iteration Agent

In this section you are to implement an agent that solves the environment, using Value Iteration

In [77]:
class ValueAgent(Agent):
  def __init__(self,env,discount_factor,theta):
    super().__init__(env,discount_factor)
    #theta is an approximation error threshold
    self.theta = theta
    self.V = np.random.rand(self.env.shape[0], self.env.shape[1])
    #set terminal state to 0
    self.V[-1,-1] = 0
  
  def act(self, state): 
    #here choose action that would bring us to state with highest value
    values=[]
    for i in range(self.env.nA):
      v=0
      for j in range(len(self.env.P[state][i])):
        prob, next_state, reward, done = self.env.P[state][i][j]
        next_position = np.unravel_index(next_state, self.env.shape)
        v += prob*(reward + self.gamma*self.V[next_position])
      values.append(v)
    
    action = np.argmax(values)
    if (type(action)==np.array): print (action)
    return action

   

  def iterate(self):
    while(True):
      delta = 0.0
      for state in range(self.env.nS-1):
        position = np.unravel_index(state, self.env.shape)
        v = self.V[position]
        action = self.act(state)
        value = 0
        for j in range(len(self.env.P[state][action])):
          prob, next_state, reward, done = self.env.P[state][action][j]
          next_position = np.unravel_index(next_state, self.env.shape)
          value += prob * (reward + self.gamma*self.V[next_position])
        self.V[position] = value
        delta = max([delta, np.abs(v-self.V[position])])
      print(delta)
      if (delta < self.theta):
        print(delta)
        break

np.set_printoptions(precision=3, linewidth=200)

agent = ValueAgent(env,0.99,0.001)
print(agent.V)
#perform value iteration
agent.iterate()
#evaluate agent and plot relevant qualities
episode_reward=agent.evaluate()
print("Episode return {}".format(episode_reward))

print(agent.V)

[[0.476 0.571 0.513 0.335 0.743 0.247 0.739 0.995 0.87  0.832 0.03  0.899]
 [0.29  0.227 0.829 0.887 0.814 0.712 0.498 0.345 0.842 0.165 0.627 0.349]
 [0.094 0.943 0.962 0.671 0.552 0.714 0.202 0.049 0.846 0.651 0.645 0.225]
 [0.23  0.206 0.154 0.136 0.153 0.876 0.216 0.877 0.379 0.233 0.405 0.   ]]
2.667789823092024
1.499011267147722
1.484021154476245
1.4691809429314824
1.4544891335021677
1.352347158321468
1.3388236867382535
1.325435449870871
1.3121810953721615
1.2990592844184388
1.1747365045306069
0.904243227670408
0.8952007953937038
0.8862487874397669
0.8640486796390174
0.0
0.0
Episode return -13
[[-13.125 -12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97 ]
 [-12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97   -1.99 ]
 [-11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97   -1.99   -1.   ]
 [-12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -1.      0.  

# Policy Iteration Agent
Follow the same procedure for implementing a policy iteration agent

In [79]:
class PolicyAgent(Agent):
  def __init__(self,env,discount_factor,theta):
    super().__init__(env,discount_factor)
    #theta is an approximation error threshold
    self.theta = theta
    self.V = np.random.rand(self.env.shape[0], self.env.shape[1])
    #set terminal state to 0
    self.V[-1,-1] = 0
    self.policy = np.random.randint(4, size=self.env.shape)
  
  def act(self, state): 
    #here choose action that would bring us to state with highest value
    values=[]
    for i in range(self.env.nA):
      v=0
      for j in range(len(self.env.P[state][i])):
        prob, next_state, reward, done = self.env.P[state][i][j]
        next_position = np.unravel_index(next_state, self.env.shape)
        v += prob * (reward + self.gamma*self.V[next_position])
      values.append(v)
    
    action = np.argmax(values)
    if (type(action)==np.array): print (action)
    return action

   

  def evaluate_policy(self):
    while(True):
      delta = 0.0
      for state in range(self.env.nS-1):
        position = np.unravel_index(state, self.env.shape)
        v = self.V[position]
        action = self.policy[position]
        value = 0
        for j in range(len(self.env.P[state][action])):
          prob, next_state, reward, done = self.env.P[state][action][j]
          next_position = np.unravel_index(next_state, self.env.shape)
          value += prob * (reward + self.gamma*self.V[next_position])
        self.V[position] = value
        delta = max([delta, np.abs(v-self.V[position])])
      #print(delta)
      if (delta < self.theta):
        #print(delta)
        break

  def improve(self):
    policy_stable = True
    #for i in range(200):
    while(True):
      policy_stable = True
      for state in range(self.env.nS-1):
        position = np.unravel_index(state, self.env.shape)
        old_action = self.policy[position]
        self.policy[position] = self.act(state)
        if not (self.policy[position] == old_action):
          policy_stable = False
      if policy_stable:
        break
      else:
        self.evaluate_policy()





agent = PolicyAgent(env,0.99,0.001)
print(agent.V)

print(agent.policy)
#perform value iteration
agent.improve()
#evaluate agent and plot relevant qualities
episode_reward=agent.evaluate()
print("Episode return {}".format(episode_reward))
np.set_printoptions(precision=3, linewidth=200)
print(agent.V)

print(agent.policy)

[[0.689 0.788 0.312 0.997 0.065 0.295 0.505 0.212 0.324 0.971 0.08  0.23 ]
 [0.599 0.01  0.092 0.852 0.212 0.353 0.206 0.345 0.397 0.755 0.717 0.149]
 [0.186 0.202 0.738 0.981 0.657 0.297 0.499 0.812 0.032 0.438 0.139 0.73 ]
 [0.77  0.891 0.183 0.651 0.525 0.851 0.255 0.392 0.639 0.996 0.88  0.   ]]
[[0 3 2 3 2 1 3 1 1 1 3 0]
 [3 0 3 1 2 3 1 3 3 2 1 0]
 [3 3 1 1 2 0 3 0 1 3 3 1]
 [0 2 2 1 1 3 2 0 2 1 0 0]]
Episode return -13
[[-13.125 -12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97 ]
 [-12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97   -1.99 ]
 [-11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97   -1.99   -1.   ]
 [-12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -1.      0.   ]]
[[1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [0 0 0 0 0 0 0 0 0 0 1 0]]


In [80]:
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv
import copy

class NonDCliffWalkingEnv(CliffWalkingEnv):
  def __init__(self):
    super(NonDCliffWalkingEnv, self).__init__()
  
  def _calculate_transition_prob(self, current, delta):
    """
    Determine the outcome for an action. Transition Prob is always 1.0.
    :param current: Current position on the grid as (row, col)
    :param delta: Change in position for transition
    :return: (1.0, new_state, reward, done)
    """
    transitions = []
    terminal_state = (self.shape[0] - 1, self.shape[1] - 1)
    deltas = [delta, [2*delta[0], 2*delta[1]]]
    zero_index = delta.index(0)
    delta_off = copy.copy(delta)
    delta_off[zero_index] = 1
    deltas.append(copy.copy(delta_off))
    delta_off[zero_index] = -1
    deltas.append(delta_off)
    probabilities = [0.85,0.05,0.05,0.05]

    
    for d,p in zip(deltas, probabilities):
      new_position = np.array(current) + np.array(d)
      new_position = self._limit_coordinates(new_position).astype(int)
      new_state = self.start_state_index if self._cliff[tuple(new_position)] else np.ravel_multi_index(tuple(new_position), self.shape)
      reward = -100 if self._cliff[tuple(new_position)] else -1
      is_done = tuple(new_position) == terminal_state
      #print(tuple(new_position), self._cliff[tuple(new_position)])
      transitions.append((p, new_state, reward, is_done))
    
    return transitions
  

In [74]:
penv = NonDCliffWalkingEnv()
len(penv.P[33][1])

4

In [81]:
agent = ValueAgent(penv,0.99,0.001)
print(agent.V)
#perform value iteration
agent.iterate()
#evaluate agent and plot relevant qualities
episode_reward=agent.evaluate()
print("Episode return {}".format(episode_reward))

print(agent.V)

[[0.73  0.713 0.755 0.991 0.766 0.739 0.006 0.152 0.395 0.403 0.809 0.872]
 [0.197 0.632 0.583 0.153 0.231 0.704 0.908 0.471 0.28  0.971 0.037 0.231]
 [0.052 0.859 0.236 0.555 0.752 0.273 0.107 0.957 0.596 0.406 0.178 0.548]
 [0.149 0.223 0.366 0.194 0.879 0.812 0.3   0.122 0.042 0.456 0.514 0.   ]]
3.195485307266794
1.8264885663777233
1.7086905478232992
1.6039457938409836
1.5112050822321326
1.4285616351882844
1.3548277899922239
1.289043803178373
1.2308974382021711
0.9531310043923362
0.9427318156800428
0.9324878752455739
0.9223892417134429
0.9124275115706908
0.9025954662639961
0.8926118351231072
0.8823241819631686
0.8681937178627948
0.8400751102013579
0.4156893480130037
0.18851470705277507
0.0525691208825414
0.04000626839519583
0.03921831509546081
0.03878952333653274
0.038369180270020564
0.03795403485254667
0.037540632941261975
0.03711305977252621
0.03656051919224623
0.03513551090035705
0.03017640164746993
0.019116988632095655
0.007472309832273538
0.0023964026376859238
0.00166437272077

In [82]:
pagent = PolicyAgent(penv,0.99,0.001)
print(agent.V)

print(pagent.policy)
#perform value iteration
pagent.improve()
#evaluate agent and plot relevant qualities
episode_reward=pagent.evaluate()
print("Episode return {}".format(episode_reward))

print(pagent.V)

print(pagent.policy)

[[-17.381 -16.592 -15.798 -14.999 -14.196 -13.389 -12.578 -11.765 -10.949 -10.131  -9.31   -8.445]
 [-17.326 -16.486 -15.635 -14.772 -13.897 -13.009 -12.108 -11.192 -10.262  -9.31   -8.445  -7.504]
 [-18.114 -17.326 -16.486 -15.635 -14.773 -13.897 -13.009 -12.108 -11.192 -10.262  -7.504  -6.883]
 [-18.855 -18.109 -17.279 -16.436 -15.581 -14.714 -13.834 -12.941 -12.034 -11.022  -1.341   0.   ]]
[[1 2 0 1 2 2 1 3 0 0 3 2]
 [0 2 1 0 0 2 0 2 1 2 0 0]
 [1 2 1 3 2 3 0 3 0 2 1 0]
 [1 2 0 3 1 0 0 1 3 3 2 1]]
Episode return -15
[[-17.384 -16.594 -15.8   -15.001 -14.197 -13.39  -12.58  -11.766 -10.95  -10.132  -9.311  -8.446]
 [-17.328 -16.488 -15.637 -14.774 -13.899 -13.011 -12.109 -11.193 -10.263  -9.311  -8.446  -7.505]
 [-18.116 -17.328 -16.488 -15.637 -14.774 -13.899 -13.011 -12.109 -11.193 -10.263  -7.505  -6.883]
 [-18.856 -18.11  -17.28  -16.437 -15.582 -14.715 -13.835 -12.942 -12.035 -11.023  -1.341   0.   ]]
[[1 1 1 1 1 1 1 1 1 1 2 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [0 0 0 0 0 0 0 0 0 0 1 

#Monte Carlo control agent
Follow the same procedure for implementing a Monte Carlo control agent

In [13]:
#code here